In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
from google.colab import files

# Subir archivos .sav
uploaded = files.upload()

In [2]:
!pip install pyreadstat
import pyreadstat

# Cargar archivos .sav
a_2018, meta1 = pyreadstat.read_sav("2018.sav")
a_2019, meta2 = pyreadstat.read_sav("2019.sav")
a_2020, meta3 = pyreadstat.read_sav("2020.sav")
a_2021, meta4 = pyreadstat.read_sav("2021.sav")
a_2022, meta4 = pyreadstat.read_sav("2022.sav")

# Visualizar los primeros registros
print(a_2018.head())

      AÑO   MES  DIASESTANCIA  SEXO  PPERTENENCIA  EDAD  PERIODOEDA  \
0  2018.0   4.0           3.0   1.0           4.0  39.0         3.0   
1  2018.0  11.0           5.0   1.0           4.0  22.0         3.0   
2  2018.0   7.0           3.0   1.0           4.0  22.0         3.0   
3  2018.0   8.0           3.0   2.0           9.0  43.0         3.0   
4  2018.0   8.0           2.0   2.0           4.0  34.0         3.0   

   DEPTORESIDEN MUNIRESIDEN CAUFIN  CONDIEGRES  TRATARECIB  
0          23.0        2300   A010         1.0         1.0  
1          23.0        2300   A010         1.0         1.0  
2          20.0        2001   A010         1.0         1.0  
3          19.0        1901   A010         1.0         1.0  
4          18.0        1804   A010         1.0         1.0  


In [3]:
# Concatenar filas
d_total = pd.concat([a_2018,a_2019,a_2020,a_2021, a_2022], ignore_index=True)

# Verificar resultado
print(d_total.head())


      AÑO   MES  DIASESTANCIA  SEXO  PPERTENENCIA  EDAD  PERIODOEDA  \
0  2018.0   4.0           3.0   1.0           4.0  39.0         3.0   
1  2018.0  11.0           5.0   1.0           4.0  22.0         3.0   
2  2018.0   7.0           3.0   1.0           4.0  22.0         3.0   
3  2018.0   8.0           3.0   2.0           9.0  43.0         3.0   
4  2018.0   8.0           2.0   2.0           4.0  34.0         3.0   

   DEPTORESIDEN MUNIRESIDEN CAUFIN  CONDIEGRES  TRATARECIB  
0          23.0        2300   A010         1.0         1.0  
1          23.0        2300   A010         1.0         1.0  
2          20.0        2001   A010         1.0         1.0  
3          19.0        1901   A010         1.0         1.0  
4          18.0        1804   A010         1.0         1.0  


In [4]:
#Convirtiendo CAUFIN a factor
# Convertir con One-Hot Encoding
#dt_one_hot = pd.get_dummies(d_total, columns=['CAUFIN'])
d_total['Cat_caufin'] = d_total['CAUFIN'].astype('category')

# Ver el DataFrame con columnas One-Hot
print(d_total)

            AÑO   MES  DIASESTANCIA  SEXO  PPERTENENCIA  EDAD  PERIODOEDA  \
0        2018.0   4.0           3.0   1.0           4.0  39.0         3.0   
1        2018.0  11.0           5.0   1.0           4.0  22.0         3.0   
2        2018.0   7.0           3.0   1.0           4.0  22.0         3.0   
3        2018.0   8.0           3.0   2.0           9.0  43.0         3.0   
4        2018.0   8.0           2.0   2.0           4.0  34.0         3.0   
...         ...   ...           ...   ...           ...   ...         ...   
1208567  2022.0   1.0        9999.0   1.0           4.0  82.0         3.0   
1208568  2022.0   3.0        9999.0   1.0           9.0  84.0         3.0   
1208569  2022.0   1.0        9999.0   2.0           4.0  86.0         3.0   
1208570  2022.0  10.0        9999.0   2.0           4.0  87.0         3.0   
1208571  2022.0   1.0        9999.0   1.0           4.0  90.0         3.0   

         DEPTORESIDEN MUNIRESIDEN CAUFIN  CONDIEGRES  TRATARECIB Cat_caufin

In [5]:
#Ver la columna CAUFIN
print(d_total['CAUFIN'])

0          A010
1          A010
2          A010
3          A010
4          A010
           ... 
1208567    Z988
1208568    Z988
1208569    Z988
1208570    Z988
1208571    Z988
Name: CAUFIN, Length: 1208572, dtype: object


In [6]:
import pandas as pd
# Ver el número de filas
print("Número de filas:", len(d_total))

# Ver el número de columnas
print("Número de columnas:", len(d_total.columns))

# Método alternativo que muestra ambos a la vez
print("Forma del dataset (filas, columnas):", d_total.shape)

Número de filas: 1208572
Número de columnas: 13
Forma del dataset (filas, columnas): (1208572, 13)


In [7]:
#Aleatoriedad
d_total = d_total.sample(frac = 1, random_state = 42).reset_index(drop=True)

In [18]:
#Selección de la data
X = d_total[['SEXO','DIASESTANCIA','DEPTORESIDEN']]
y = d_total['Cat_caufin']

In [21]:
#Datos de entrenamiento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train = X_train.replace(' ', np.nan).dropna().astype('float32')
# Initialize LabelEncoder
label_encoder = LabelEncoder()
# Fit and transform y_train
#Fit on all unique values to avoid unseen labels during transform
all_unique_labels = pd.concat([y_train[X_train.index], y_test[X_test.index]]).unique()
label_encoder.fit(all_unique_labels)
y_train = label_encoder.transform(y_train[X_train.index]).astype('float32')

X_test = X_test.replace(' ', np.nan).dropna().astype('float32')
# Transform y_test using the same encoder
y_test = label_encoder.transform(y_test[X_test.index]).astype('float32')

In [13]:
#Densidad de las capas, relu: función de activación de la capa (cambia valores negativos a positivos), introduce no linealidades
#sigmoid salida que representa la probabilidad de pertenencia a una de las clases
model = Sequential()
model.add(Dense(20, input_dim = 3, activation = 'relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [22]:
#Entrenamiento
model.fit(X_train, y_train, epochs = 50, batch_size = 200, validation_data=(X_test, y_test))


Epoch 1/50
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 2.4543e-07 - loss: -824515072.0000 - val_accuracy: 4.1371e-06 - val_loss: -10107454464.0000
Epoch 2/50
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 1.6353e-06 - loss: -18385016832.0000 - val_accuracy: 4.1371e-06 - val_loss: -54425088000.0000
Epoch 3/50
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 2.2870e-06 - loss: -73872310272.0000 - val_accuracy: 4.1371e-06 - val_loss: -148984348672.0000
Epoch 4/50
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 6.8580e-07 - loss: -184619302912.0000 - val_accuracy: 4.1371e-06 - val_loss: -307581812736.0000
Epoch 5/50
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 3.1650e-06 - loss: -362847272960.0000 - val_accuracy: 4.1371e-06 - val_loss: -547721641984.0000
Epoch 6/50
4835/4835 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 2.0066e-06 - loss: -623919300608.0000 - val_accuracy: 4.1371e-06 - val_loss: -881284612096.0000
Epoch 7/50
4835/4835 ━━━━━━━━

In [23]:
loss, acc = model.evaluate(X_test, y_test)
print(acc*100)

7554/7554 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 2.5849e-06 - loss: -325441281851392.0000
0.00041371035877091344
